In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.5/660.5 kB 5.0 MB/s eta 0:00:00


In [ ]:
import os
import xml.etree.ElementTree as ET


In [ ]:
#extract data from words.xml
def extract_data_from_xml(file_path):
  xml_path = os.path.join(file_path,'words.xml')
  tree = ET.parse(xml_path)
  root = tree.getroot()

  img_paths = []
  img_labels = []
  img_sizes = []
  bboxes = []

  for image in root:
    bbs_of_img = []
    labels_of_img = []

    for bbs in image.findall('taggedRectangles'):
      for bb in bbs:
        # check non-alphabet and non-number
        if not bb[0].text.isalnum():
          continue

        if ' ' in bb[0].text.lower() or ' ' in bb[0].text.lower():
          continue

        bbs_of_img.append([
            float(bb.attrib['x']),
            float(bb.attrib['y']),
            float(bb.attrib['width']),
            float(bb.attrib['height'])
        ])
        labels_of_img.append(bb[0].text.lower())


        img_path = os.path.join(file_path,image[0].text)
        img_paths.append(img_path)
        img_sizes.append((int(image[1].attrib['x']), int(image[1].attrib['y'])))
        bboxes.append(bbs_of_img)
        img_labels.append(labels_of_img)

  return img_paths, img_sizes, img_labels, bboxes

In [ ]:
dataset_dir = '/content/drive/MyDrive/SceneTrialTrain'
img_paths,img_sizes,img_labels,bboxes = extract_data_from_xml(dataset_dir)

In [ ]:
img_paths

['/content/drive/MyDrive/SceneTrialTrain/apanar_06.08.2002/IMG_1261.JPG',
 '/content/drive/MyDrive/SceneTrialTrain/apanar_06.08.2002/IMG_1261.JPG',
 '/content/drive/MyDrive/SceneTrialTrain/apanar_06.08.2002/IMG_1261.JPG',
 '/content/drive/MyDrive/SceneTrialTrain/apanar_06.08.2002/IMG_1261.JPG',
 '/content/drive/MyDrive/SceneTrialTrain/apanar_06.08.2002/IMG_1261.JPG',
 '/content/drive/MyDrive/SceneTrialTrain/apanar_06.08.2002/IMG_1261.JPG',
 '/content/drive/MyDrive/SceneTrialTrain/apanar_06.08.2002/IMG_1261.JPG',
 '/content/drive/MyDrive/SceneTrialTrain/apanar_06.08.2002/IMG_1261.JPG',
 '/content/drive/MyDrive/SceneTrialTrain/apanar_06.08.2002/IMG_1261.JPG',
 '/content/drive/MyDrive/SceneTrialTrain/apanar_06.08.2002/IMG_1261.JPG',
 '/content/drive/MyDrive/SceneTrialTrain/apanar_06.08.2002/IMG_1261.JPG',
 '/content/drive/MyDrive/SceneTrialTrain/apanar_06.08.2002/IMG_1263.JPG',
 '/content/drive/MyDrive/SceneTrialTrain/apanar_06.08.2002/IMG_1265.JPG',
 '/content/drive/MyDrive/SceneTrialTra

In [ ]:
# convert to yolov8 format
# example yolov8_labels format : 0 0.1 0.2 0.3 0.4

def convert_to_yolov8_format(img_paths,img_sizes,bboxes):
  yolov8_data = []
  for img_path,img_size,bbox in zip(img_paths,img_sizes,bboxes):
    yolov8_labels = []
    img_width,img_height = img_size
    for bb in bbox:
      x,y,w,h = bb
      x = (x + w/2)/img_width
      y = (y + h/2)/img_height
      w = w/img_width
      h = h/img_height
      class_id = 0

      label = f"{class_id} {x} {y} {w} {h}"
      yolov8_labels.append(label)
    yolov8_data.append((img_path,yolov8_labels))
  return yolov8_data


In [ ]:
class_labels = ['text']
yolov8_data = convert_to_yolov8_format(
    img_paths,
    img_sizes,
    bboxes
)

In [ ]:
#save data
import shutil
def save_data(data, src_img_dir, save_dir):
    # Create folder if not exists
    os.makedirs(save_dir, exist_ok=True)

    # Make images and labels folder
    os.makedirs(os.path.join(save_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(save_dir, 'labels'), exist_ok=True)

    for image_path, yolov8_labels in data:
        # Copy image to images folder
        shutil.copy(
            os.path.join(src_img_dir, image_path),
            os.path.join(save_dir, 'images')
        )

        # Save labels to labels folder
        image_name = os.path.basename(image_path)
        image_name = os.path.splitext(image_name)[0]

        with open(os.path.join(save_dir, 'labels', f"{image_name}.txt"), 'w') as f:
            for label in yolov8_labels:
                f.write(f"{label}\n")

In [ ]:
from sklearn.model_selection import train_test_split
seed = 0
val_size = 0.2
test_size = 0.125
is_shuffle = True

train_data, test_data = train_test_split(
    yolov8_data,
    test_size=val_size,
    random_state=seed,
    shuffle=is_shuffle
)

test_data, val_data = train_test_split(
    test_data,
    test_size=test_size,
    random_state=seed,
    shuffle=is_shuffle
)

save_yolo_data_dir = 'yolo_data'
os.makedirs(save_yolo_data_dir, exist_ok=True)

save_train_dir = os.path.join(save_yolo_data_dir, 'train')
save_val_dir = os.path.join(save_yolo_data_dir, 'val')
save_val_dir = os.path.join(save_yolo_data_dir, 'val')
save_test_dir = os.path.join(save_yolo_data_dir, 'test')

save_data(train_data, dataset_dir, save_train_dir)
save_data(test_data, dataset_dir, save_val_dir)
save_data(val_data, dataset_dir, save_test_dir)

In [ ]:
import yaml

data_yaml = {
    'path': '/content/yolo_data',
    'train': 'train/images',
    'test': 'test/images',
    'val': 'val/images',
    'nc': 1,
    'names': class_labels
}

yolo_yaml_path = os.path.join(save_yolo_data_dir, 'data.yml')

with open(yolo_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

In [ ]:
#Training
from ultralytics import YOLO
yolo_yaml_path = '/content/yolo_data/data.yml'
model = YOLO('yolov8s.yaml').load('yolov8s.pt')




                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

100%|██████████| 21.5M/21.5M [00:00<00:00, 339MB/s]

Transferred 355/355 items from pretrained weights


In [ ]:
epochs = 200
imgsz = 1024
results = model.train(
    data=yolo_yaml_path,
    epochs=epochs,
    imgsz=imgsz,
    project='models',
    name='yolov8/detect/train'
)

Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/content/yolo_data/data.yml, epochs=200, patience=50, batch=16, imgsz=1024, save=True, save_period=-1, cache=False, device=None, workers=8, project=models, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format

100%|██████████| 755k/755k [00:00<00:00, 66.6MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 168MB/s]


WARNING ⚠️ NMS time limit 0.550s exceeded
AMP: checks passed ✅


...

    200/200      10.2G     0.4291     0.2843     0.8295         67       1024: 100%|██████████| 14/14 [00:09<00:00,  1.48it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.10it/s]

                   all        118        761      0.992      0.988      0.994      0.928



200 epochs completed in 0.846 hours.
Optimizer stripped from models/yolov8/detect/train/weights/last.pt, 22.6MB
Optimizer stripped from models/yolov8/detect/train/weights/best.pt, 22.6MB

Validating models/yolov8/detect/train/weights/best.pt...
Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:05<00:00,  1.42s/it]


                   all        118        761      0.992      0.988      0.994      0.928
Speed: 0.4ms preprocess, 9.8ms inference, 0.0ms loss, 6.1ms postprocess per image
Results saved to models/yolov8/detect/train


In [ ]:
from ultralytics import YOLO

model_path = '/content/models/yolov8/detect/train/weights/best.pt'
model = YOLO(model_path)

metrics = model.val()


Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /content/yolo_data/val/labels.cache... 118 images, 0 backgrounds, 0 corrupt: 100%|██████████| 118/118 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:09<00:00,  1.21s/it]


                   all        118        761      0.991      0.988      0.994      0.928
Speed: 1.0ms preprocess, 25.9ms inference, 0.0ms loss, 10.0ms postprocess per image
Results saved to runs/detect/val


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.4 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import random
import cv2
import timm
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torchvision
from torch.nn import functional as F
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

In [ ]:
def split_bounding_boxes(img_paths, img_labels, bboxes, save_dir):
    os.makedirs(save_dir, exist_ok=True)

    count = 0
    labels = []  # List to store labels

    for img_path, img_label, bbs in zip(img_paths, img_labels, bboxes):
        img = Image.open(img_path)

        for label, bb in zip(img_label, bbs):
            # Crop image
            cropped_img = img.crop((bb[0], bb[1], bb[0] + bb[2], bb[1] + bb[3]))

            # Filter out if 90% of the cropped image is black or white
            if np.mean(cropped_img) < 35 or np.mean(cropped_img) > 220:
                continue

            if cropped_img.size[0] < 10 or cropped_img.size[1] < 10:
                continue

            # Save image
            filename = f"{count:06d}.jpg"
            cropped_img.save(os.path.join(save_dir, filename))

            new_img_path = os.path.join(save_dir, filename)

            label = new_img_path + '\t' + label

            labels.append(label)  # Append label to the list

            count += 1

    print(f"Created {count} images")

    # Write labels to a text file
    with open(os.path.join(save_dir, 'labels.txt'), 'w',encoding='utf-8') as f:
        for label in labels:
            f.write(f"{label}\n")

save_dir = 'ocr_dataset'
split_bounding_boxes(img_paths, img_labels, bboxes, save_dir)

Created 11314 images


In [ ]:
#read data
root_dir = save_dir

img_paths = []
labels = []
with open(os.path.join(root_dir,'labels.txt'),'r',encoding='utf-8') as f:
  for label in f :
    labels.append(label.strip().split('\t')[1])
    img_paths.append(label.strip().split('\t')[0])

print(f"total image {len(img_paths)}")

total image 11314


In [ ]:
#letters = [char.split(".")[0].lower() for char in labels]
i = 0
for char in labels:
  print(char)
  i += 1
  if i == 5:
    break

self
adhesive
address
labels
36


In [ ]:
letters = [char.lower() for char in labels]

In [ ]:
letters = "".join(letters)
print(letters)


...ésacksadthecafésacksadthecafésacksadthecafésacksadtheagentsagentsestateestatesaxonssaxonsagentsagentsestateestatesaxonssaxonsagentsagentsestateestatesaxonssaxonsagentsagentsestateestatesaxonssaxonsagentsagentsestateestatesaxonssaxonsagentsagentsestateestatesaxonssaxonssaversboxstationeryboxstationerydell8000dellinspiron8000dellinspiron8000dellinspironfallinghungrykneesmyonfallinghungrykneesmyonfallinghungrykneesmyonfallinghungrykneesmyonfallinghungrykneesmyonadaptercanoncompactpoweradaptercanoncompactpoweradaptercanoncompactpoweradaptercanoncompactpowercanonpowershotcanonpowershotdailyfarnboroughflyingjulyprogramme200228airshowdailyfarnboroughflyingjulyprogramme200228airshowdailyfarnboroughflyingjulyprogramme200228airshowdailyfarnboroughflyingjulyprogramme200228airshowdailyfarnboroughflyingjulyprogramme200228airshowdailyfarnboroughflyingjulyprogramme200228airshowdailyfarnboroughflyingjulyprogramme200228airshowdailyfarnboroughflyingjulyprogramme200228airshowbaesystemsbaesystemsacitie

In [ ]:
letters = sorted(list(set(list(letters))))
letters

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é',
 'ñ']

In [ ]:
chars = "".join(letters)
chars

'0123456789abcdefghijklmnopqrstuvwxyzéñ'

In [ ]:
blank_char = '-'
chars += blank_char
vocab_size = len(chars)

In [ ]:
print(vocab_size)

39


In [ ]:
char_to_idx = {char: idx + 1 for idx, char in enumerate(sorted(chars))}
idx_to_char = {index: char for char, index in char_to_idx.items()}

In [ ]:
max_label_len = max([len(label) for label in labels])
print(max_label_len)

14


In [ ]:
max_label_len = max([len(label) for label in labels])

def encode(label, char_to_idx, max_label_len):
    encoded_labels = torch.tensor(
        [char_to_idx[char] for char in label],
        dtype=torch.int32
    )
    label_len = len(encoded_labels)
    lengths = torch.tensor(
        label_len,
        dtype=torch.int32
    )
    padded_labels = F.pad(
        encoded_labels,
        (0, max_label_len - label_len),
        value=0
    )

    return padded_labels, lengths

In [ ]:
def decode(encoded_sequences, idx_to_char, blank_char='-'):
    decoded_sequences = []

    for seq in encoded_sequences:
        decoded_label = []
        for idx, token in enumerate(seq):
            if token != 0:
                char = idx_to_char[token.item()]
                if char != blank_char:
                    decoded_label.append(char)

        decoded_sequences.append(''.join(decoded_label))

    return decoded_sequences

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((100, 420)),
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
        transforms.Grayscale(num_output_channels=1),
        transforms.GaussianBlur(3),
        transforms.RandomAffine(degrees=1, shear=1),
        transforms.RandomPerspective(distortion_scale=0.3, p=0.5, interpolation=3),
        transforms.RandomRotation(degrees=2),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ]),
    'val': transforms.Compose([
        transforms.Resize((100, 420)),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ]),
}

In [ ]:
seed = 0
val_size = 0.2
test_size = 0.125
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(
    img_paths, labels,
    test_size=val_size,
    random_state=seed,
    shuffle=is_shuffle
)

X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train,
    test_size=test_size,
    random_state=seed,
    shuffle=is_shuffle
)

In [ ]:
class STRDataset(Dataset):
    def __init__(
        self,
        X, y,
        char_to_idx,
        max_label_len,
        label_encoder=None,
        transform=None
    ):
        self.transform = transform
        self.img_paths = X
        self.labels = y
        self.char_to_idx = char_to_idx
        self.max_label_len = max_label_len
        self.label_encoder = label_encoder

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        label = self.labels[idx]
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        if self.label_encoder:
            encoded_label, label_len = self.label_encoder(
                label,
                self.char_to_idx,
                self.max_label_len
            )
        return img, encoded_label, label_len

In [ ]:
train_dataset = STRDataset(
    X_train, y_train,
    char_to_idx=char_to_idx,
    max_label_len=max_label_len,
    label_encoder=encode,
    transform=data_transforms['train']
)

val_dataset = STRDataset(
    X_val, y_val,
    char_to_idx=char_to_idx,
    max_label_len=max_label_len,
    label_encoder=encode,
    transform=data_transforms['val']
)

test_dataset = STRDataset(
    X_test, y_test,
    char_to_idx=char_to_idx,
    max_label_len=max_label_len,
    label_encoder=encode,
    transform=data_transforms['val']
)

train_batch_size = 32
test_batch_size = 8

train_loader = DataLoader(
    train_dataset,
    batch_size=train_batch_size,
    shuffle=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=test_batch_size,
    shuffle=False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=test_batch_size,
    shuffle=False
)

In [ ]:
class CRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_layers, dropout=0.2, unfreeze_layers=3):
        super(CRNN, self).__init__()

        backbone = timm.create_model(
            'resnet101',
            in_chans=1,
            pretrained=True
        )
        modules = list(backbone.children())[:-2]
        modules.append(nn.AdaptiveAvgPool2d((1, None)))
        self.backbone = nn.Sequential(*modules)

        # Unfreeze the last few layers
        for parameter in self.backbone[-unfreeze_layers:].parameters():
            parameter.requires_grad = True

        self.mapSeq = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

        self.lstm = nn.LSTM(
            512, hidden_size,
            n_layers, bidirectional=True, batch_first=True,
            dropout=dropout if n_layers > 1 else 0
        )
        self.layer_norm = nn.LayerNorm(hidden_size * 2)

        self.out = nn.Sequential(
            nn.Linear(hidden_size * 2, vocab_size),
            nn.LogSoftmax(dim=2)
        )

    def forward(self, x):
        x = self.backbone(x)
        x = x.permute(0, 3, 1, 2)
        x = x.view(x.size(0), x.size(1), -1)  # Flatten the feature map
        x = self.mapSeq(x)
        x, _ = self.lstm(x)
        x = self.layer_norm(x)
        x = self.out(x)
        x = x.permute(1, 0, 2)  # Based on CTC

        return x

In [ ]:
hidden_size = 256
n_layers = 3
dropout_prob = 0.2
unfreeze_layers = 3
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = CRNN(
    vocab_size=vocab_size,
    hidden_size=hidden_size,
    n_layers=n_layers,
    dropout=dropout_prob,
    unfreeze_layers=unfreeze_layers
).to(device)

model.safetensors:   0%|          | 0.00/179M [00:00<?, ?B/s]

In [ ]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    losses = []
    with torch.no_grad():
        for inputs, labels, labels_len in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            labels_len = labels_len.to(device)

            outputs = model(inputs)
            logits_lens = torch.full(
                size=(outputs.size(1),),
                fill_value=outputs.size(0),
                dtype=torch.long
            ).to(device)

            loss = criterion(outputs, labels, logits_lens, labels_len)
            losses.append(loss.item())

    loss = sum(losses) / len(losses)

    return loss

def fit(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs):
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        batch_train_losses = []

        model.train()
        for idx, (inputs, labels, labels_len) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            labels_len = labels_len.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            logits_lens = torch.full(
                size=(outputs.size(1),),
                fill_value=outputs.size(0),
                dtype=torch.int32
            ).to(device)

            loss = criterion(outputs, labels, logits_lens, labels_len)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(
                model.parameters(),
                5
            )
            optimizer.step()

            batch_train_losses.append(loss.item())

        train_loss = sum(batch_train_losses) / len(batch_train_losses)
        train_losses.append(train_loss)

        val_loss = evaluate(
            model, val_loader,
            criterion, device
        )
        val_losses.append(val_loss)

        print(f'EPOCH {epoch + 1}:\tTrain loss: {train_loss:.4f}\tVal loss: {val_loss:.4f}')

        scheduler.step()

    return train_losses, val_losses

In [ ]:
epochs = 35
lr = 0.001
weight_decay = 1e-5
scheduler_step_size = int(epochs * 0.6)

criterion = nn.CTCLoss(blank=char_to_idx[blank_char], zero_infinity=True)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=scheduler_step_size,
    gamma=0.1
)

In [ ]:
train_losses, val_losses = fit(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    scheduler,
    device,
    epochs
)

EPOCH 1:	Train loss: 3.4030	Val loss: 2.6575
EPOCH 2:	Train loss: 1.7965	Val loss: 0.6801
EPOCH 3:	Train loss: 0.6141	Val loss: 0.5803
EPOCH 4:	Train loss: 0.3603	Val loss: 0.1410
EPOCH 5:	Train loss: 0.2489	Val loss: 0.1291
EPOCH 6:	Train loss: 0.1966	Val loss: 0.1488
EPOCH 7:	Train loss: 0.2252	Val loss: 0.1024
EPOCH 8:	Train loss: 0.1571	Val loss: 0.0775
EPOCH 9:	Train loss: 0.1315	Val loss: 0.0648
EPOCH 10:	Train loss: 0.0971	Val loss: 0.0409
EPOCH 11:	Train loss: 0.1378	Val loss: 0.0718
EPOCH 12:	Train loss: 0.1378	Val loss: 0.0593
EPOCH 13:	Train loss: 0.1506	Val loss: 0.1218
EPOCH 14:	Train loss: 0.1195	Val loss: 0.0542
EPOCH 15:	Train loss: 0.1019	Val loss: 0.0433
EPOCH 16:	Train loss: 0.1082	Val loss: 0.0777
EPOCH 17:	Train loss: 0.1132	Val loss: 0.0352
EPOCH 18:	Train loss: 0.0690	Val loss: 0.0324
EPOCH 19:	Train loss: 0.1721	Val loss: 0.0566
